## Árvore Radix

O código abaixo ilustra a inclusão, busca e exclusão em uma árvore Radix. Existem implementações com diferentes tamanhos de r (número de filhos). Neste notebook são usados nós com 26 filhos.

Cada nó que possui apenas 1 filho é fusionado com o seu nó pai. Uma descrição informal pode ser encontrada no link: https://en.wikipedia.org/wiki/Radix_tree

In [1]:
#include <iostream>
using namespace std;

#define ALFABETO 26
#define A_ASCII 65;

Estrutura para uma árvore Radix, com grau máximo 26. O grau máximo é dado pelo valor do alfabeto especificado acima. O nó folha possui a chave.

O radical tem tamanho fixo 10, e a chave é um ponteiro para char. Estas escolhas poderiam ser modificadas de acordo com a aplicação.

Quando o nó representa uma palavra, a chave inteira é armzenada. Existem outras implementações possíveis, por exemplo, com grau máximo = 2. Neste caso, o tamanho do nó é menor, porém a altura da árvore será maior.

In [2]:
struct tNo {
    tNo *p [ALFABETO];
    char radical[10];
    char *chave;
    int tamanho;    
} 

Percurso da árvore em **ORDEM** pela chave. Imprime o radical associado a cada nó. Quando é um nó folha (tem uma chave associada), imprime também a chave.

In [3]:
void emordem(tNo *no){    
    int j = 0;
    if (no != NULL) {
       for (int i=0; i < ALFABETO; i++){
           if (no->p[i] != NULL) {
               cout<< no->p[i]->radical;
               if (no->p[i]->chave[0] != '\0')
                   cout << "(" << no->p[i]->chave << ")";
                cout <<'.';
               emordem(no->p[i]);
           }
       }
    }
}

Conta o número de nós da árvore, incluindo o nó raiz.

In [4]:
int contaNos(tNo *no){
   int nos = 0; 
   if (no != NULL) {
       for (int i=0; i < ALFABETO ;i++){
           if (no->p[i] != NULL)
               nos += contaNos(no->p[i]);
       }
       nos += 1;
   }
   return nos;
}

Calcula a altura da árvore. A altura da árvore será também o comprimento da maior chave (palavra) encontrada.

In [5]:
int altura (tNo *no) {
   int hTotal = -1; 
   if (no != NULL) {
       int h=-1;
       for (int i=0; i < ALFABETO ;i++){
           if (no->p[i] != NULL){
              h = altura(no->p[i]);
               if (h > hTotal)
                   hTotal = h;
           }
       }
       hTotal += 1;
   }
   return hTotal;
}

Função que converte um caracter de entrada em um número inteiro. Esta função converte todas as letras para maíusculas e diminui do valor ASCII da letra 'A', que é 65. O valor está definido na constante A_ASCII;

In [6]:
int char_to_index(char chr){
    char c = toupper(chr);
    return c - A_ASCII;
}

Função que retorna um token de uma determinada string, separada por espaços. Usada para processar a string de entrada de dados.

In [7]:
char* get_token(const char *str, int *indice){    
    char * token = (char *) malloc(sizeof(char)*20) ;
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return token;
}

Busca em árvore Radix. Percorre os componentes das chaves baseados na string de entrada, e retorna o nó folha encontrado.

Imprime o radical dos nós percorridos.

In [8]:
tNo *busca (tNo *no, const char *chave, int inicio) {
    int nivel, indice_char, j;
    tNo *noMeio = NULL, *noFilho = NULL;
    int tamanho_chave = strlen(chave);
    char radix[15]; 
    int i=0;
    indice_char = char_to_index(chave[inicio]);
    noFilho = no->p[indice_char];
    if (noFilho == NULL) //nao encontrou a chave
        return NULL;
    else {
        cout << noFilho->radical << ".";
        while (chave[i+inicio] == noFilho->radical[i])
            i++;        
        if (i < noFilho->tamanho) // não encontrou
            return NULL;
        else {
            //diminiui 1 do i, pois já está posicionado no próximo caracter.
            if ((i-1) == noFilho->tamanho && (i - 1 + inicio) == tamanho_chave) //encontrou palavra                
               return noFilho;
            else 
               return busca(noFilho, chave, i + inicio );
        }
    }
}

Função auxiliar para imprimir a chave de um nó.

In [9]:
void imprime(const char *str, tNo *no){
    cout << "||" << str << " ";
    if (no!=NULL) cout << "["<< no->chave<<"]";
    else cout << "não";
    cout << " encontrado\n";
}

Função para inicialização do nó, com alocação de memória, atribuição de valores NULL para os ponteiros e atribuição dos valores para o radical de cada nó e para a chave, caso exista.

In [12]:
tNo *criaNo (const char *radix, int inicio, int fim, const char * chave){
    tNo *no = (tNo *)malloc (sizeof (tNo));
    int i, j, tamanho_chave = strlen(chave) ;
    for (int j=0; j < ALFABETO; j++) //zera os ponteiros
       no->p[j] = NULL;
    no->chave = (char *)malloc(sizeof(char) * 20); //aloca a chave com 20 caracteres.
    no->chave[0] = '\0';
    for (i=0; i < fim; i ++) //seta um valor para o radical
        no->radical[i] = radix[i+inicio];    
    no->radical[i] = '\0';
    no->tamanho = strlen(no->radical); //fim-inicio;
    if (inicio + fim == tamanho_chave){
        for (j=0; j < tamanho_chave; j ++)
            no->chave[j] = chave[j];
        no->chave[j] = '\0';
    }
    return no;
}

Funcão para inclusão de uma nova chave na árvore Radix, dado um nó raiz. 

Percorre a árvore de acordo até um nó folha e inclui uma nova chave. Caso necessário, faz um **split** de um nó já existente e ajusta os valores dos radicais e ponteiros. O split é feito na descida. Existem implementações que faze o split na subida.

O nó final armazena a chave inteira.

In [13]:
void inclui (tNo *no, const char *chave, int inicio){
    int nivel, indice_char, j;
    tNo *noMeio = NULL, *noFilho = NULL;
    int tamanho_chave = strlen(chave);
    char radix[15]; 
    int i=0;
    indice_char = char_to_index(chave[inicio]);
    noFilho = no->p[indice_char];
    if (noFilho == NULL){ //nao existe subarvore, incluir noh
        no->p[indice_char] = criaNo(chave, inicio, tamanho_chave-inicio, chave);
        i = tamanho_chave;
    } else {
        while (chave[i+inicio] == noFilho->radical[i]){
            i++;        
        }
        if (i < noFilho->tamanho){ // split do nó
            noMeio = criaNo(noFilho->radical, 0, i, chave); //novo nó será o nó do meio
            no->p[indice_char] = noMeio;
            noFilho->tamanho = noFilho->tamanho - i;
            for (j = 0; j < noFilho->tamanho ; j++)
                noFilho->radical[j] = noFilho->radical[j + i];
            noFilho->radical[j] = '\0';
            noMeio->p[char_to_index(noFilho->radical[0])] = noFilho;
            if (i != tamanho_chave) 
              inclui(noMeio,chave,i+inicio);                  
        }
        else {
            if ((i-1) == noFilho->tamanho && (i - 1 + inicio) == tamanho_chave) {//encontrou palavra
                return;
            } else { //chama próximo nó (se for palavra maior que a existente, irá incluir novo no)
                for (int k = i; k < strlen(chave); k ++)
                    cout << chave [k];
                inclui(noFilho, chave, i + inicio );
            }
        }
    }
}

Faz a leitura da string de entrada, separa cada token com espaços e chama a função de inclusão.

In [14]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0;
    raiz = criaNo("\0",0,0, "\0");
    while (str[i]!='\0')
        inclui (raiz, get_token(str, &i),0);
    return raiz;        
}


In [15]:
void iniciaprograma(){
    tNo *raiz = NULL;
    
    char entrada[] = "casado casa carro bola busca\0";
    raiz=montaarvore(entrada); 
    
    cout << "\nPercurso em ordem: ";
    emordem(raiz); cout << "\n";
    
    cout << "\nTotal de nós: " << contaNos(raiz);
    cout << "\nAltura da árvore: " << altura(raiz) << "\n\n";
    
     imprime (" busca carra", busca(raiz, "carra",0));
     imprime (" busca ", busca(raiz, "cargo",0));
     imprime (" busca ", busca(raiz, "bola",0));
     imprime (" busca ", busca(raiz, "carta",0));
    
}

In [16]:
iniciaprograma();


Percurso em ordem: b.ola(bola).usca(busca).ca.rro(carro).sa(casa).do(casado).

Total de nós: 8
Altura da árvore: 3

ca.rro.|| busca carra não encontrado
ca.rro.|| busca  não encontrado
b.ola.|| busca  [bola] encontrado
ca.rro.|| busca  não encontrado
